## EE 460J Lab 3 Report

### Lab Group Members: Tatsushi Matsumoto, Nick Taylor, Matthew Withey

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pdfminer.high_level
import requests
from bs4 import BeautifulSoup
from IPython.display import display
import re
import os

### Problem 2

In [2]:
# funciton to scrape all pdfs
def download_all_pdfs(url_name):
    url = requests.get(url_name)
    soup = BeautifulSoup(url.text, 'html.parser')
    pdf_links = soup.find_all('a')

    count = 0
    for link in pdf_links:
        if ('.pdf' in link.get('href', [])):
            count += 1
            response = requests.get(link.get('href'))
            pdf = open('pdfs/pdf' + str(count) + '.pdf', 'wb')
            pdf.write(response.content)
            pdf.close() 

    print('All ' + str(count) + ' pdfs downloaded')

url = 'http://proceedings.mlr.press/v70/'
# comment this out if you don't want to redownload on every run
# download_all_pdfs(url)
# comment this out if you don't want to redownload on every run

In [3]:
def number_of_files(path):
    path, dirs, files = next(os.walk(path))
    file_count = len(files)
    return file_count

def extract_text_from_pdfs():
    for i in range(number_of_files('pdfs/')):
        try:
            text = pdfminer.high_level.extract_text('pdfs/pdf' + str(i+1) + '.pdf')
            text_file = open('pdfs_text/text' + str(i+1) + '.txt', 'w', encoding='utf-8')
            text_file.write(text)
            text_file.close()
        except:
            pass
    
    print('All text files downloaded')
# comment this out if you don't want to redownload on every run
# extract_text_from_pdfs()
# comment this out if you don't want to redownload on every run

In [4]:
def get_sorted_word_count():
    frequency = {}
    for i in range(number_of_files('pdfs_text/')):
        try:
            document_text = open('pdfs_text/text' + str(i+1) + '.txt', 'r', encoding='utf-8')
            text_string = document_text.read().lower()
            match_pattern = re.findall(r'\b[a-z]{1,25}\b', text_string)
            for word in match_pattern:
                count = frequency.get(word,0)
                frequency[word] = count + 1
        except:
            pass
        
    df = pd.DataFrame(list(frequency.items()), columns=['word', 'count'])
    df = df.sort_values(by='count', ascending=False)
    return df

df_words = get_sorted_word_count()
df_top = df_words.head(10)
display(df_top)
print('Top 10 words in all ICML papers: ' + str(df_top['word'].to_numpy()))

,word,count
28,the,205708
222,cid,129073
30,of,102346
59,and,87829
18,a,79736
117,in,70450
26,to,66162
51,is,55902
79,for,51423
16,we,50899


Top 10 words in all ICML papers: ['the' 'cid' 'of' 'and' 'a' 'in' 'to' 'is' 'for' 'we']


In [5]:
def estimate_entropy():
    total_words = df_words['count'].sum()
    prob = df_words['count'].div(total_words)
    df_words['probability'] = prob

    random_word = df_words.sample(weights='probability')
    p = random_word.iloc[0]['probability']
    q = 1 - p
    display(random_word)

    H = -1*(p*np.log2(p)+q*np.log2(q))
    print('Word: ' + str(random_word.iloc[0]['word']))
    print('p: ' + str(p))
    print('q: ' + str(q))
    print('Estimated entropy: ' + str(H))

estimate_entropy()

,word,count,probability
1773,tional,541,0.000138


Word: tional
p: 0.00013804365853969713
q: 0.9998619563414604
Estimated entropy: 0.001969218081923932
